<a href="https://colab.research.google.com/github/KC-Night/DS-2002-Assignments/blob/main/Finance_API_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import pandas as pd
import json
import requests

Task **1**

In [ ]:
# Prompt the user to input two or more stock symbols
stocks = input("Enter two or more stock symbols separated by commas: ").replace(" ", "")

In [ ]:
#Use the API to fetch data for the given symbols
apikey="ENTER API KEY HERE"

url = "https://yfapi.net/v6/finance/quote"

querystring = {"symbols":stocks.strip()}

headers = {
    'x-api-key': apikey
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

In [ ]:
print(json.dumps(response.json(), indent=3))

In [21]:
#Extract and display only the Stock Ticker, Company Name, and Current Market Price.
df = pd.DataFrame(response.json()["quoteResponse"]["result"])[
        ["symbol", "shortName", "regularMarketPrice"]
    ]
df["regularMarketPrice"] = df["regularMarketPrice"].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")

for _, row in df.iterrows():
        print(f"Stock Ticker: {row['symbol']}, Company: {row['shortName']}, Current Market Price: {row['regularMarketPrice']}")

Stock Ticker: RBLX, Company: Roblox Corporation, Current Market Price: $59.78
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.36


Task 2.1

In [46]:
# Allow the user to choose a module from the Quote Summary Endpoint, Find: 52 Week High, 52 Week Low, Return on Assets (ROA), Display as Dataframe
stock = input("Enter the stock symbol: ")

def get_quote_summary_data(choice, stock):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{stock}"
    querystring = {"modules": "price,summaryDetail,financialData"}
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    result = data.get("quoteSummary", {}).get("result", [{}])[0]
    if not result:
        return "No data found for the given stock symbol."

    module_name = "Invalid Choice"
    value = "N/A"

    if choice == "52 Week High":
        module_name, value = "52-Week High", result.get("summaryDetail", {}).get("fiftyTwoWeekHigh", {}).get("raw", "N/A")
    elif choice == "52 Week Low":
        module_name, value = "52-Week Low", result.get("summaryDetail", {}).get("fiftyTwoWeekLow", {}).get("raw", "N/A")
    elif choice == "Return on Assets (ROA)":
        roa_data = result.get("financialData", {}).get("returnOnAssets", {})

        roa = roa_data.get("raw", None)
        if roa is not None:
            value = f"{roa:.2%}"
        else:
            value = "N/A"
        module_name = "Return on Assets (ROA)"
    else:
        return "Invalid choice. Please choose from 52 Week High, 52 Week Low, or Return on Assets (ROA)."

    df = pd.DataFrame({module_name: [value]}, index=[stock])
    return df

print("Choose an option: 52 Week High, 52 Week Low, Return on Assets (ROA)")
user_choice = input("Enter your choice: ")

result_df = get_quote_summary_data(user_choice, stock)
print(result_df)

Enter the stock symbol: AAPL
Choose an option: 52 Week High, 52 Week Low, Return on Assets (ROA)
Enter your choice: Return on Assets (ROA)
     Return on Assets (ROA)
AAPL                 22.52%


Task 2.2

In [ ]:
#Find Current Trending Stocks , Their Actual Name and Ticker and Current Price as well as their 52 high and low

url= "https://yfapi.net/v1/finance/trending/US"

querystring = {"region":"US"}

headers = {
    'x-api-key': apikey
}

response = requests.get(url, headers=headers, params=querystring)

print(json.dumps(response.json(), indent=3))


In [ ]:
trending_url = "https://yfapi.net/v1/finance/trending/US"
quote_url = "https://yfapi.net/v6/finance/quote"
headers = {'x-api-key': apikey}

def fetch_trending_stocks():
    response = requests.get(trending_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        quotes = data.get("finance", {}).get("result", [])
        if quotes:
            return [stock["symbol"] for stock in quotes[0].get("quotes", [])]
    return []

def fetch_stock_details(symbols):
    querystring = {"symbols": ",".join(symbols)}
    response = requests.get(quote_url, headers=headers, params=querystring)
    if response.status_code == 200:
        data = response.json()
        return data.get("quoteResponse", {}).get("result", [])
    return []

trending_symbols = fetch_trending_stocks()
if trending_symbols:
    stock_details = fetch_stock_details(trending_symbols)
    if stock_details:
        stock_list = [
            {
                "Stock Name": stock.get("shortName", "N/A"),
                "Ticker": stock.get("symbol", "N/A"),
                "Current Price (USD)": stock.get("regularMarketPrice", "N/A"),
                "52-Week High (USD)": stock.get("fiftyTwoWeekHigh", "N/A"),
                "52-Week Low (USD)": stock.get("fiftyTwoWeekLow", "N/A")
            }
            for stock in stock_details
        ]

        df = pd.DataFrame(stock_list)
        print(df)
    else:
        print("No detailed stock data found.")
else:
    print("No trending stocks found.")
